In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.layers import Input, Dense
from keras.models import Model
from keras.regularizers import l1

In [35]:
df = pd.read_csv("ackley.csv")
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,target
0,-5.809773,25.168431,-18.354026,-13.612737,-26.288529,5.304547,19.829286,26.805632,-30.343183,-20.082952,21.403924
1,-20.405254,7.894409,-27.188741,16.283305,-29.748012,1.006791,-18.400917,1.454824,6.986963,8.408095,21.075111
2,24.324245,24.312616,-13.219045,10.569770,-1.207372,27.741237,27.645784,-2.837093,11.937643,14.460268,21.344547
3,19.572219,31.280122,21.343705,-8.335016,29.910004,27.525442,-17.659962,16.890195,10.853367,-3.716901,21.550162
4,-23.117079,25.999806,1.717129,-30.424215,30.471974,26.845881,-13.640445,8.750917,26.574302,23.548366,21.720362


In [36]:
# descriptive statistics of the dataset
descriptive_stats = df.describe()
descriptive_stats

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,target
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.097740,0.087019,0.039849,-0.150259,0.170307,0.025833,0.251800,-0.033383,-0.031175,-0.295300,21.097617
std,18.406686,18.503033,18.549034,18.601066,18.411179,18.552889,18.513458,18.501518,18.562738,18.529469,0.428040
min,-31.982935,-31.996448,-31.997701,-31.998780,-31.999805,-31.990230,-31.997495,-31.992971,-31.999592,-31.998929,17.302517
25%,-15.953672,-15.948880,-16.240481,-16.694236,-15.741310,-16.006710,-16.145195,-15.948280,-16.104392,-16.205878,20.887439
50%,-0.126936,0.074381,0.044752,-0.182541,0.184243,-0.365499,0.477923,-0.196748,-0.256268,-0.298176,21.159950
75%,16.119616,16.026557,16.204796,15.965724,16.047624,16.122016,16.363389,16.121237,16.209138,15.723710,21.387934
max,31.998509,31.990013,31.993752,31.997751,31.998163,31.999150,31.986743,31.990696,31.987639,31.986088,22.024739


In [37]:
X_std = StandardScaler().fit_transform(df)

In [38]:
# Number of features in the input
input_dim = 11
# Set the dimensionality of the encoded (hidden) representation
encoded_dim = 10  # Changing this to 10 for 10 hidden nodes

# Define the input layer
input_layer = Input(shape=(input_dim,))

# Define the encoded (hidden) layer
encoded = Dense(encoded_dim, activation='relu', activity_regularizer=l1(1e-5))(input_layer)

# Define the decoded layer
decoded = Dense(input_dim, activation='sigmoid')(encoded)

# Define the autoencoder model
autoencoder = Model(input_layer, decoded)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Assuming X_std is your standardized dataset
# Train the autoencoder
autoencoder.fit(X_std, X_std, epochs=50, batch_size=256, shuffle=True, validation_split=0.2)


Epoch 1/50
32/32 [==============================] - 1s 6ms/step - loss: 1.2512 - val_loss: 1.2285
Epoch 2/50
32/32 [==============================] - 0s 2ms/step - loss: 1.1951 - val_loss: 1.1743
Epoch 3/50
32/32 [==============================] - 0s 2ms/step - loss: 1.1436 - val_loss: 1.1235
Epoch 4/50
32/32 [==============================] - 0s 2ms/step - loss: 1.0947 - val_loss: 1.0753
Epoch 5/50
32/32 [==============================] - 0s 2ms/step - loss: 1.0485 - val_loss: 1.0304
Epoch 6/50
32/32 [==============================] - 0s 2ms/step - loss: 1.0062 - val_loss: 0.9902
Epoch 7/50
32/32 [==============================] - 0s 2ms/step - loss: 0.9687 - val_loss: 0.9551
Epoch 8/50
32/32 [==============================] - 0s 2ms/step - loss: 0.9363 - val_loss: 0.9252
Epoch 9/50
32/32 [==============================] - 0s 2ms/step - loss: 0.9087 - val_loss: 0.8999
Epoch 10/50
32/32 [==============================] - 0s 2ms/step - loss: 0.8853 - val_loss: 0.8784
Epoch 11/50
32/32 [

In [39]:
# Create a separate model for the encoded (hidden) layer to access the reduced dimensions
encoder = Model(input_layer, encoded)

# To get the encoded (reduced dimension) output for a given input data (e.g., X_std)
encoded_output = encoder.predict(X_std)

313/313 [==============================] - 0s 658us/step


In [40]:
# Print the shape of the encoded output
print("Shape of encoded output:", encoded_output.shape)

Shape of encoded output: (10000, 10)


In [41]:
# Alternatively, print only the first few rows of the encoded output
print("First few rows of Encoded Output:")
print(encoded_output[:])  # Adjust the number as needed

First few rows of Encoded Output:
[[3.6360233  0.         0.41626978 ... 4.3395963  0.         3.416203  ]
 [1.659308   1.7155526  2.242031   ... 1.1393654  0.4458226  2.9344146 ]
 [0.         0.         0.7060803  ... 1.7897782  0.         1.841846  ]
 ...
 [1.5199821  0.         0.         ... 2.1403928  1.4417713  1.3573983 ]
 [2.4249969  0.         0.28744304 ... 0.         2.5644107  1.000809  ]
 [0.36023986 2.723522   3.2055786  ... 0.5813017  3.075306   2.5387151 ]]


In [42]:
# Assuming encoded_output is your array from the encoder model
feature_indices = np.argmax(encoded_output, axis=1)
print(feature_indices)

[7 6 9 ... 7 6 2]


In [43]:
threshold = 0.1
thresholded_output = np.where(encoded_output > threshold, 1, 0)

In [44]:
print(thresholded_output.shape)

(10000, 10)


In [45]:
# Removing duplicate rows
unique_rows = np.unique(thresholded_output, axis=0)
print(len(unique_rows))

866


In [48]:
def find_overlapping_rows(unique_rows, overlap_threshold):
    n_rows = unique_rows.shape[0]
    overlapping_groups = []

    for i in range(n_rows):
        for j in range(i+1, n_rows):
            # Count the number of matching elements
            similarity = np.sum(unique_rows[i] == unique_rows[j])
            if similarity >= overlap_threshold:
                # Convert arrays to tuples for comparison
                group = (tuple(unique_rows[i]), tuple(unique_rows[j]))
                overlapping_groups.append(group)

    # Select unique groups until we have 10 of them
    unique_groups = []
    for group in overlapping_groups:
        if group not in unique_groups:
            unique_groups.append(group)
            if len(unique_groups) == 10:
                break

    return unique_groups


In [49]:
overlap_threshold = 3  # Define how many elements need to match for rows to be considered overlapping
overlapping_rows = find_overlapping_rows(unique_rows, overlap_threshold)
print(overlapping_rows)

[((0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 0, 0, 1)), ((0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 0, 1, 0)), ((0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1, 0, 1)), ((0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 1, 1, 1, 1)), ((0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 1, 0, 0, 0, 0)), ((0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 1, 0, 1, 0, 1)), ((0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 1, 0, 1, 1, 0)), ((0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 1, 0, 1, 1, 1)), ((0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 1, 1, 0, 1, 0)), ((0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 1, 1, 0, 1, 1))]
